In [ ]:
#!pip install --upgrade pip setuptools wheel
#!python -m pip install opencv-python
#!pip install matplotlib
#!pip freeze
#!pip install opencv-python-headless
#!pip install scikit-image
#!pip install -U scikit-fuzzy
#!pip install pandas
#!pip install scikit-learn
#!pip install tqdm==4.40.0

In [ ]:
from matplotlib import pyplot as plt
import cv2
import os
import random
import pandas as pd
import numpy as np
from os.path import join
from skimage import measure
from skimage.feature import blob_dog, blob_log, blob_doh
import skfuzzy as fuzz
import library

### IMAGE PROCESSING - PREPROCESSING

In [ ]:
# from bansal paper laplacian of gaussian + top hat morphology
# NOT FINISHED!!!!!!
def hairRemoval_LoG_THM(matrix, ksize = 3, kernel_size = 17, filterSize = (11, 11)):
    # 1. grayscale
    gray = cv2.cvtColor(matrix, cv2.COLOR_BGR2GRAY)
    
    # 2. laplacian of gaussian
    # steps are blurring,
    #           laplacian,
    #           zero_crossing,
    #           threshold zero crossing to extract strong edges
    ddepth = cv2.CV_16S
    blur = cv2.GaussianBlur(gray, (ksize, ksize), 0)
    lp = cv2.Laplacian(blur, ddepth, ksize=ksize)
    # converting output back to uint8
    log = cv2.convertScaleAbs(lp)

    # 3. closing and dilation
    kernel = cv2.getStructuringElement(1,(kernel_size,kernel_size))
    close = cv2.morphologyEx(log, cv2.MORPH_CLOSE, kernel)
    dil = cv2.dilate(close, kernel)
    
    # 4. top-hat transformation   
    kernel_th = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, 
                                          filterSize)
    
    tophat = cv2.morphologyEx(dil, 
                              cv2.MORPH_TOPHAT,
                              kernel_th)
    # 5. otsu
    #thresh = cv2.threshold(sharp, 0, 255, cv2.THRESH_OTSU )[1]
    # 6. erosion and closing
    # 7. impainting
    
    return log, close, dil, tophat

# MAIN

First pipeline with 

- hair removal
- segmentation with kmeans
- extracting asymmetry from the images

In [ ]:
FOLDER_DIR = "/home/emily/Desktop/CAD/challenge1/train"

# img = cv2.imread(join(FOLDER_DIR,"nevus", "nev{}.jpg".format("03178")))
img = cv2.imread(join(FOLDER_DIR, "others", "{}.jpg".format("ack00322")))
# 00656 02056 01831 00547 02237  02462 02655  03178 03284   04438  05209 bcc00220 bcc00242
# 04762 04176 cute
# 00246 05464 02758 02462 02379 shity
# 03863 finger
# others ack00291 ack00363 bcc00001 bc00103 ack00028 ack00180 ack00322 bcc00221 bcc00271 bkl00160 ack00021
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

nevus = np.array(img_rgb)
plt.imshow(img_rgb)
plt.title('original')
plt.show()

output_bh = library.hair_removal_BH(img)

plt.imshow(cv2.cvtColor(output_bh, cv2.COLOR_BGR2RGB))
plt.title('preprocessing blackhat')
plt.show()


mask = segmentation_kmeans(output_bh)

plt.imshow(mask, cmap='gray')
plt.title('mask')
plt.show()

# print('mask.shape ', mask.shape)

# bg = output_bh.copy()
# fg = output_bh.copy()

# bg[mask == 0]
# fg[mask != 0]


# plt.imshow(bg)
# plt.show()


# plt.imshow(fg)
# plt.show()


# colormean = mean_std(bg)

asy = asymmetry(mask, False)
print('asymmetry value: ', asy)



Test with 1000 images to generate asymmetry

### GENERATING FEATURES

In [ ]:
import library
import texture_features
import color_features
import glcm_features
import hair_removal
import pandas as pd
import numpy as np
import cv2
import os

samples = library.get_sample(path = "/home/name/Desktop/CAD/challenge1/train", amount=3000)

dictF = {}
features = pd.DataFrame()
count = 0
flag = True
for sample in samples:
    print('count ', count)
    count += 1
    img = cv2.imread(sample)
    output_bh = hair_removal.hair_remove(img, 17, 8)
    
    dictF['name'] = sample
    dictF['label'] = (0 if 'nevus' in sample else 1 )
    
    # color features
    colors = color_features.extract_color_features(output_bh)
    dictF.update(colors)
    
    # glcm features
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    distances = [1]
    colorspaces = ['rgb', 'hsv', 'lab', 'ycc', 'gray']
    
    for cs in colorspaces:
        glcm = glcm_features.get_glcm(output_bh, angles, distances, cs)
        dictF.update(glcm)
    
    
    # lbp features
    lbp = texture_features.extract_lbp(output_bh, 1, 8)
    dictF.update(lbp)
    
    # orb features
    # orb = texture_features.extract_orb(output_bh, 64)
    # dictF.update(orb)
    
    features = features.append(dictF, ignore_index=True)
    
    library.writeFeatures(features,
                  flag,
                  os.path.join('/home',
                             'name',
                             'Desktop',
                             'CAD'),
                  'features_train_shaver_3000.csv')
    
    flag = False
    features = pd.DataFrame()
    dictF.clear()


/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/

/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/name/.local/

/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/li

/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/li

/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/li

...

/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/li

/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/

/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/name/.local/

/tmp/ipykernel_13288/2568142318.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(dictF, ignore_index=True)


count  304


# FEATURES EXTRACTION

## ABCD Rule

commonly used feature extraction method from dermoscopic images is based on the ABCD rule of dermatoscopu nachbar et al 1994.  this method consider 4 criteria: 

- asymmetry: if you draw a like through the half of the mole and the halfs are different. its asymetrical
- border structure
- color variation
- diameter of skin lesion.

inspiration for code from

https://github.com/sohum2002/melanoma-detection/blob/master/melanoma/feature_extraction/asymmetry_index.py
https://github.com/hadikhanhk786/melanoma-detection-python/blob/master/src/Lesion.py
https://github.com/RCharradi/Melanoma-detection-from-dermoscopy-images-

 ### ASYMMETRY


In [ ]:
# https://github.com/sohum2002/melanoma-detection/blob/master/melanoma/feature_extraction/asymmetry_index.py

def checkOverlap(shape1, shape2):
    #Find the accuracy of symmetry
    all_pixels = 0.
    correct = 0.
    wrong = 0.

    for i in range(shape1.shape[0]):
        for j in range(shape1.shape[1]):

            curr_pixel1 = (shape1[i][j])
            curr_pixel2 = (shape2[i][j])

            if(curr_pixel1 or curr_pixel2):
                all_pixels += 1
                if(curr_pixel1 and curr_pixel2):
                    correct += 1
                else:
                    wrong += 1

    return correct, wrong, all_pixels

def asymmetry(mask, flag):
    
    # 0. variables
    x = []
    y = []
    top  = np.zeros((mask.shape[0], mask.shape[1]), dtype=bool)
    left = np.zeros((mask.shape[0], mask.shape[1]), dtype=bool)
    bottom = np.zeros((mask.shape[0], mask.shape[1]), dtype=bool)
    right = np.zeros((mask.shape[0], mask.shape[1]), dtype=bool)
    
    # 1. find the center of the image
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if (mask[i][j] != 1):
                x.append(j)
                y.append(i)
    
    # centroid = ( gray.shape[0] / 2, gray.shape[1] / 2) # need to change if segmentation maskk
    centroid = (sum(x) / len(x), sum(y) / len(y))
    print('centroid ', centroid)
    
    # print(np.unique(mask))
    
    # 2. split top/down images
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if (mask[i][j] < 1):
                if (i < centroid[1]):
                    top[i][j] = True
    if flag:
        plt.imshow(top)
        plt.title('top')
        plt.show()
    
    # 4. split left/right images
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if (mask[i][j] < 0.95):
                if (j < centroid[0]):
                    left[i][j] = True
    if flag:
        plt.imshow(left)
        plt.title('left')
        plt.show()
    
    # doing for flip top/down images
    flipped_ud = np.flipud(mask)
    
    for i in range(flipped_ud.shape[0]):
        for j in range(flipped_ud.shape[1]):
            if(flipped_ud[i][j] < 0.95):
                if(i < centroid[1]):
                    bottom[i][j] = True
    
    if flag:
        plt.imshow(bottom)
        plt.title('bottom')
        plt.show()
    
    # doing for flip left/right to get the right part
    flipped_lr = np.fliplr(mask)
    
    # performing splitting for top/down images
    for i in range(flipped_lr.shape[0]):
        for j in range(flipped_lr.shape[1]):
            if(flipped_lr[i][j] < 0.95):
                if(j < centroid[0]):
                    right[i][j] = True
    if flag:
        plt.imshow(right)
        plt.title('right')
        plt.show()        
        
    correct_TB, wrong_TB, all_TB = checkOverlap(top, bottom)
    correct_LR, wrong_LR, all_LR = checkOverlap(left, right)
    
    if flag:
        print(correct_TB)
        print(all_TB)
        print(correct_LR)
        print(all_LR)
    
    return 1- sum([correct_TB / all_TB, correct_LR / all_LR]) / 2

result_asym = asymmetry(mask, True)

print('symmetry ', result_asym)

# MACHINE LEARNING

In [ ]:
from sklearn.model_selection import train_test_split
import library
from sklearn import tree
import pandas as pd
import os
    

classifiers = ["rf", "tree", "svm", "adaboost", "gradboost", "knn"]

train = pd.read_csv(os.path.join('/home','emily','Desktop','CAD','features_bh_colors_glcm_5562.csv'))

y = train['label']
X = train.drop(['label'], axis=1)
X = X.drop(['name'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

for classifier in classifiers:
    
    if classifier == "svm":
        clf, best_params = library.SVC_linear(X_val, y_val, cv=2)
        clf.set_params(**best_params) 
        print("### SVM ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "rf":
        clf, best_params = library.RandomForest(X_val, y_val, cv=2)
        clf.set_params(**best_params)
        print("### RF ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "tree":
        clf = tree.DecisionTreeClassifier()
        print("### TREE ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "adaboost":
        clf, best_params = library.AdaBoost(X_val, y_val)
        clf.set_params(**best_params)
        print("### ADABOOST ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "gradboost":
        clf, best_params = library.GradientBoosting(X_val, y_val)
        clf.set_params(**best_params)
        print("### GRADBOOST ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "knn":
        clf, best_params = library.knn(X_val, y_val)
        clf.set_params(**best_params)
        print("### KNN ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)